In [38]:
%%capture
!pip install transformers

In [39]:
# Wrap cels
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {white-space: pre-wrap;}
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [40]:
import numpy as np
import pandas as pd

from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

import torch

In [41]:
articles_df = pd.read_csv('drive/My Drive/!UBS/DW/data/covid_main.csv', index_col=0, dtype ={'body': str})
print(articles_df.shape)
articles_df.head()

(1933, 8)


,sha,title,abstract,publish_time,authors,url,introduction,body
31175,f056da9c64fbf00a4645ae326e8a4339d015d155,SIANN: Strain Identification by Alignment to N...,Next-generation sequencing is increasingly bei...,2014-01-10,Samuel Minot; Stephen D Turner; Krista L Ternu...,https://doi.org/10.1101/001727,There are many different methods that characte...,Metagenomic classification methods are based o...
31176,daf32e013d325a6feb80e83d15aabc64a48fae33,Spatial epidemiology of networked metapopulati...,An emerging disease is one infectious epidemic...,2014-06-04,Lin WANG; Xiang Li,https://doi.org/10.1101/003889,The term metapopulation was coined by Levins [...,Due to the ubiquity of complex systems in mode...
31177,f33c6d94b0efaa198f8f3f20e644625fa3fe10d2,Sequencing of the human IG light chain loci fr...,Germline variation at immunoglobulin gene (IG)...,2014-07-03,Corey T Watson; Karyn Meltz Steinberg; Tina A ...,https://doi.org/10.1101/006866,Antibodies are essential components of the imm...,We analyzed sequences from 17 CH17 BAC clones ...
31178,4da8a87e614373d56070ed272487451266dce919,Bayesian mixture analysis for metagenomic comm...,Deep sequencing of clinical samples is now an ...,2014-07-25,Sofia Morfopoulou; Vincent Plagnol,https://doi.org/10.1101/007476,Metagenomics can be defined as the analysis of...,We first applied metaMix on a popular benchmar...
31179,eccef80cfbe078235df22398f195d5db462d8000,Mapping a viral phylogeny onto outbreak trees ...,Developing methods to reconstruct transmission...,2014-11-11,Stephen P Velsko; Jonathan E Allen,https://doi.org/10.1101/010389,NaN,: Schematic of transmission links overlaid on ...


### Download BART pretrained model

In [42]:
tokenizer = BartTokenizer.from_pretrained('bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('bart-large-cnn')
model.cuda();

In [43]:
#Function for abstract genetation
def generate_absctract(model, tokenizer, text, max_lenght):
    inputs = tokenizer.batch_encode_plus([text], max_length=1024, return_tensors='pt')
    summary_ids = model.generate(inputs['input_ids'].cuda(), 
                                 max_length=max_lenght,
                                 early_stopping=True,
                                 do_sample=True,
                                 top_k=40,
                                 top_p=0.95).cpu()
    output = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
    return '\n'.join(output)

### Generate abstract

In [51]:
article_no = 3
original_text = articles_df.iloc[article_no]['body']
original_abstract = articles_df.iloc[article_no]['abstract']
original_abstract_len = len(original_abstract.split())

print(20*'-', 'ORIGINAL ABSTRACT', 20*'-')
print(original_abstract)
generatet_abstract = generate_absctract(model, tokenizer, original_text, original_abstract_len)
print(20*'-', 'GENERATED ABSTRACT', 20*'-')
print(generatet_abstract)

-------------------- ORIGINAL ABSTRACT --------------------
Deep sequencing of clinical samples is now an established tool for the detection of infectious pathogens, with direct medical applications. The large amount of data generated provides an opportunity to detect species even at very low levels, provided that computational tools can effectively interpret potentially complex metagenomic mixtures. Data interpretation is complicated by the fact that short sequencing reads can match multiple organisms and by the lack of completeness of existing databases, in particular for viral pathogens. This interpretation problem can be formulated statistically as a mixture model, where the species of origin of each read is missing, but the complete knowledge of all species present in the mixture helps with the individual reads assignment. Several analytical tools have been proposed to approximately solve this computational problem. Here, we show that the use of parallel Monte Carlo Markov chains 

In [52]:
original_text

'We first applied metaMix on a popular benchmark dataset, for which the community composition and the read assignment is known. We then analyzed RNA-Seq datasets from two clinical samples that were generated for diagnostic purposes. We compare our results with the ones produced by MEGAN version 5.3 and Pathoscope 2.0. Both methods are similarity-based. This property and more specifically their flexibility to work with BLASTx output, makes them better candidates for viral discovery compared to compositionbased methods. From the mixture model methods, we have chosen Pathoscope. We were also interested in comparing our results to the ones by GRAMMy, which was the first similarity-based method to use the idea of the mixture model. However, GRAMMy is designed for nucleotide-nucleotide comparisons (BLASTn), which is suboptimal for viral discovery. GRAMMy also only considers unassembled reads and requires that these are of the same fixed length. For these reasons, GRAMMy was not included in t